In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from amazoncaptcha import AmazonCaptcha
import re
import time
import numpy as np
import json
import math
import sys
from datetime import datetime, timedelta
import csv
import pandas as pd
from fuzzywuzzy import process

def getElement(browser,xpath,attr=None):
    wait = WebDriverWait(browser, 2)  # 2 seconds (adjust as needed)
    element = None
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH,xpath)))
        if attr is None:
            return element  # Stop the loop if element is found
        elif (attr=='text'):
            element = element.text
            if (element == ""):
                raise Exception("text is empty")
        else:
            element = element.get_attribute(attr)
    except Exception as e:
        # print("didn't find any element")
        pass
    return element

C:\Users\Dell\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
# Daraz

browser = webdriver.Chrome()
browser.get('https://www.daraz.pk/')

input_search = getElement(browser,"//input[@type='search']")
input_search.clear()
input_search.send_keys("iphone 14 pro")
search_button =browser.find_element(By.XPATH, "//button[@class='search-box__button--1oH7']")
search_button.click()
sleep(2)
products = browser.find_elements(By.XPATH,"//div[@class='gridItem--Yd0sa']")
products_data = {}
for product in products:
    title =  getElement(product,".//div[@class='title--wFj93']","textContent")
    price =  getElement(product,".//span[@class='currency--GVKjl']","text")
    products_data[title] = price

csv_file = 'iphone_prices_daraz.csv'

# Writing data to CSV
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Phone Model', 'Price'])
    for model, price in products_data.items():
        writer.writerow([model, price])
print(f'Data has been written to {csv_file}')
products_data


Data has been written to iphone_prices_daraz.csv


{'Like New Phones - Used Apple iPhone X - Silver 256 GB - PTA Approved': 'Rs. 106,999',
 'Like New Phones - Used Apple iPhone SE 2nd Gen (2020) - Black 64 GB - PTA Approved': 'Rs. 70,999',
 'Like New Phones - Used Apple iPhone 8 - Gold 64 GB - PTA Approved': 'Rs. 57,999',
 'Like New Phones - Used Apple iPhone SE 2nd Gen (2020) - White 64 GB - PTA Approved': 'Rs. 70,999',
 'Daraz Like New Phones - Used Apple iPhone 11 Pro 64 GB Midnight Green - PTA Approved': 'Rs. 180,999',
 'Like New Phones - Used Apple iPhone X - Space Gray 256 GB - PTA Approved': 'Rs. 106,999',
 'iPhone 15 256GB Apple Official 1 year Mercantile Warranty': 'Rs. 436,499',
 'iPhone 15 Pro 128 GB Apple official 1 Year Mercantile Warranty - PTA APPROVED': 'Rs. 502,499',
 'Like New Phones - Used Apple iPhone X - Silver 64 GB - PTA Approved': 'Rs. 95,999',
 'Like New Phones - Used Apple iPhone X - Space Gray 64 GB - PTA Approved': 'Rs. 95,999',
 'Like New Phones - Used Apple iPhone 8 Plus - Gold 64 GB - PTA Approved': 'Rs. 

In [5]:
# Telemart
browser = webdriver.Chrome()
browser.get('https://www.telemart.pk/')

input_search = getElement(browser,"//input[@id='search-box']")
input_search.clear()
input_search.send_keys("iphone 14 pro")
search_button = browser.find_element(By.XPATH, "//button[@class='focus:outline-none flex items-center roboto-new  leading-normal bg-orange-400 bg-hover-orange-500 rounded-r-sm px-4 text-grey-dark text-xl']")
search_button.click()
sleep(4)
products = browser.find_elements(By.XPATH,"//div[@class='bg-white relative cursor-pointer p-0.5 col-span-3']")
products_data = {}
for product in products:
    title =  getElement(product,".//h4","text")
    price =  getElement(product,".//span[@class='inline-block tracking-tighter roboto-new font-normal product-title-size mt-1 text-green-600']","text")
    products_data[title] = price

csv_file = 'iphone_prices_telemart.csv'

# Writing data to CSV
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Phone Model', 'Price'])
    for model, price in products_data.items():
        writer.writerow([model, price])
print(f'Data has been written to {csv_file}')
products_data


Data has been written to iphone_prices_telemart.csv


{'Apple iPhone 14 Pro 128GB (Dual eSim) - PTA Approved - Pre Owned': 'Rs. 394,999',
 'Apple iPhone 14 Pro 256GB (Dual eSim) - PTA Approved - Pre Owned': 'Rs. 414,999',
 'Apple iPhone 14 Pro Max 256GB (Single + eSim) - PTA Approved - Pre Owned': 'Rs. 479,999',
 'Apple iPhone 14 Pro Max 128GB (Dual eSim) - PTA Approved - Pre Owned': 'Rs. 429,999',
 'Apple iPhone 14 Pro Max 256GB (Dual eSim) - PTA Approved - Pre Owned': 'Rs. 459,999',
 'Apple iPhone 14 Pro 128GB (Single + eSim) - PTA Approved - Pre Owned': 'Rs. 409,999',
 'Apple iPhone 14 Pro 256GB (Single + eSim) - PTA Approved - Pre Owned': 'Rs. 449,999',
 'Apple iPhone 14 Pro Max 128GB (Single + eSim) - PTA Approved - Pre Owned': 'Rs. 449,999',
 'Apple iPhone 14 Pro Max 512GB (Dual Esim) PTA Approved..(LLA)': 'Rs. 522,749',
 'Apple iPhone 14 Pro Max 256GB (Dual eSIM) PTA Approved (LLA)': 'Rs. 499,999',
 'Apple iPhone 14 Pro 512GB (Dual eSIM) PTA Approved (LLA)': 'Rs. 492,299',
 'Apple iPhone 14 Pro 128GB (Dual eSIM) - PTA Approved (LLA

### Loading Data and Cleaning using Pandas

In [22]:
df1=pd.read_csv('iphone_prices_daraz.csv')
df1.head()

,Phone Model,Price
0,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999"
1,Like New Phones - Used Apple iPhone SE 2nd Gen...,"Rs. 70,999"
2,Like New Phones - Used Apple iPhone 8 - Gold 6...,"Rs. 57,999"
3,Like New Phones - Used Apple iPhone SE 2nd Gen...,"Rs. 70,999"
4,Daraz Like New Phones - Used Apple iPhone 11 P...,"Rs. 180,999"


In [23]:
df2=pd.read_csv('iphone_prices_telemart.csv')
df2.head()

,Phone Model,Price
0,Apple iPhone 14 Pro 128GB (Dual eSim) - PTA Ap...,"Rs. 394,999"
1,Apple iPhone 14 Pro 256GB (Dual eSim) - PTA Ap...,"Rs. 414,999"
2,Apple iPhone 14 Pro Max 256GB (Single + eSim) ...,"Rs. 479,999"
3,Apple iPhone 14 Pro Max 128GB (Dual eSim) - PT...,"Rs. 429,999"
4,Apple iPhone 14 Pro Max 256GB (Dual eSim) - PT...,"Rs. 459,999"


In [24]:
df= df1.drop_duplicates()
daraz_data=df.dropna()
daraz_data.head()

,Phone Model,Price
0,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999"
1,Like New Phones - Used Apple iPhone SE 2nd Gen...,"Rs. 70,999"
2,Like New Phones - Used Apple iPhone 8 - Gold 6...,"Rs. 57,999"
3,Like New Phones - Used Apple iPhone SE 2nd Gen...,"Rs. 70,999"
4,Daraz Like New Phones - Used Apple iPhone 11 P...,"Rs. 180,999"


In [25]:
df3= df2.drop_duplicates()
telemart_data=df3.dropna()
telemart_data.head()

,Phone Model,Price
0,Apple iPhone 14 Pro 128GB (Dual eSim) - PTA Ap...,"Rs. 394,999"
1,Apple iPhone 14 Pro 256GB (Dual eSim) - PTA Ap...,"Rs. 414,999"
2,Apple iPhone 14 Pro Max 256GB (Single + eSim) ...,"Rs. 479,999"
3,Apple iPhone 14 Pro Max 128GB (Dual eSim) - PT...,"Rs. 429,999"
4,Apple iPhone 14 Pro Max 256GB (Dual eSim) - PT...,"Rs. 459,999"


In [38]:
def fuzzy_search(row):
    matches = process.extractOne(search_term, row['Phone Model'])
    return pd.Series([matches[0], matches[1]])

search_term = 'Apple iPhone 14 Pro Max' 

matches_df1 = daraz_data.apply(fuzzy_search, axis=1)
matches_df2 = telemart_data.apply(fuzzy_search, axis=1)

best_match_df1 = matches_df1.loc[matches_df1[1].idxmax()]
best_match_df2 = matches_df2.loc[matches_df2[1].idxmax()]


print(search_term)
if best_match_df1[1] >= best_match_df2[1]:
    print("Recommendation: Buy from Telemart")
else:
    print("Recommendation: Buy from Daraz")

Apple iPhone 14 Pro Max
Recommendation: Buy from Telemart
